In [ ]:
pip install -U langchain langchain-community langchain-core langchain-chroma langchain-huggingface pypdf python-dotenv sentence-transformers llama-cpp-python deep-translator jira

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from deep_translator import GoogleTranslator
from dotenv import load_dotenv
from jira import JIRA
from langchain.docstore.document import Document

# --- 1. CONFIGURACIÓN INICIAL Y CONEXIÓN A JIRA ---
load_dotenv()
JIRA_SERVER = os.getenv("JIRA_SERVER")
JIRA_EMAIL = os.getenv("JIRA_EMAIL")
JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")

if not JIRA_SERVER or not JIRA_EMAIL or not JIRA_API_TOKEN:
    print("Error: The JIRA credentials were not loaded. Please check your .env file.")
    exit()

try:
    jira_options = {'server': JIRA_SERVER}
    jira_connection = JIRA(options=jira_options, basic_auth=(JIRA_EMAIL, JIRA_API_TOKEN))
except Exception as e:
    print(f"Error connecting to Jira: {e}")
    exit()

pdf_path = "/content/2020-Scrum-Guide-US.pdf"
model_path = "/content/drive/My Drive/Lexi-Llama-3-8B-Uncensored_Q4_K_M.gguf"  # Updated path
persist_directory = "/chroma_db"

# --- 2. FUNCIONES DE EXTRACCIÓN Y CREACIÓN DE LA BASE DE CONOCIMIENTO UNIFICADA ---

def obtener_documentos_de_jira():
    """
    Obtiene issues de un proyecto de Jira y los convierte en documentos de texto.
    """
    try:
        proyecto = "My RAG app"
        tipo_issue = "Story"
        jql_query = f'project = "{proyecto}" AND issuetype = "{tipo_issue}" ORDER BY created DESC'
        issues = jira_connection.search_issues(jql_query, maxResults=50)

        documentos_jira = []
        for issue in issues:
            texto_issue = f"ID: {issue.key}\nResumen: {issue.fields.summary}\nDescripción: {issue.fields.description}\nEstado: {issue.fields.status.name}\nTipo: {issue.fields.issuetype.name}"
            documentos_jira.append(Document(page_content=texto_issue))

        return documentos_jira
    except Exception as e:
        print(f"Error al obtener datos de Jira: {e}")
        return []

def crear_base_de_conocimiento_unificada():
    """
    Carga documentos del PDF y datos de Jira, los combina y crea una única base de datos vectorial.
    """
    print("Cargando la Guía de Scrum...")
    loader = PyPDFLoader(pdf_path)
    documents_pdf = loader.load()

    print("Obteniendo documentos de Jira...")
    documents_jira = obtener_documentos_de_jira()

    all_documents = documents_pdf + documents_jira
    if not all_documents:
        print("No se encontraron documentos en ninguna fuente. No se creará la base de conocimiento.")
        return None

    print(f"Número de documentos cargados: {len(all_documents)}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(all_documents)

    embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
    print("Base de conocimiento unificada (PDF y Jira) creada y persistida.")
    return db

# --- 3. FUNCIÓN PARA CONSULTAR LA BASE DE CONOCIMIENTO ---

def limpiar_respuesta(texto):
    """
    Limpieza básica del texto de respuesta del LLM.
    """
    # Expresión regular para eliminar caracteres de control y patrones de texto basura del modelo
    cleaned_text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', texto)
    cleaned_text = re.sub(r'llama_kv_cache_unified.*', '', cleaned_text, flags=re.DOTALL)
    cleaned_text = re.sub(r'\s+y el.*', '', cleaned_text, flags=re.DOTALL)
    return cleaned_text.strip()

def consultar_base_de_conocimiento(query):
    """
    Traduce la pregunta del usuario, busca en la base de datos y genera una respuesta con el LLM.
    """
    try:
        translated_query = GoogleTranslator(source='es', target='en').translate(query)
        print(f"Pregunta traducida (a inglés): {translated_query}")
    except Exception as e:
        print(f"Error al traducir la pregunta: {e}. Usando la pregunta original.")
        translated_query = query

    embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    retriever = db.as_retriever(search_kwargs={"k": 2})

    llm = LlamaCpp(
        model_path=model_path,
        n_gpu_layers=-1,
        n_batch=256,
        n_ctx=4096,
        callback_manager=None,
        verbose=False,
        temperature=0.1,
        max_tokens=512,
        stop=["<|eot_id|>", "Question:", "Pregunta original:", "Ingresa tu pregunta (o 'salir' para terminar):"]
    )

    # El prompt ahora sugiere usar saltos de línea para mejorar la legibilidad
    template = """Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer only the user's question directly. Use line breaks to make the answer easier to read.

{context}

Question: {question}
Helpful Answer:"""

    qa_prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": qa_prompt}
    )

    result_en = qa.invoke({"query": translated_query})

    if isinstance(result_en, dict) and 'result' in result_en:
        result_text_en = result_en['result']
    else:
        result_text_en = str(result_en)

    try:
        result_text_en = limpiar_respuesta(result_text_en)
        result_es = GoogleTranslator(source='en', target='es').translate(result_text_en)
    except Exception as e:
        print(f"Error al traducir la respuesta: {e}. Mostrando respuesta en inglés.")
        result_es = result_text_en

    print(f"Pregunta original: {query}")
    print(f"Respuesta (en español): {result_es}")

# --- 4. PUNTO DE ENTRADA DEL SCRIPT ---

if __name__ == "__main__":
    if not os.path.exists(persist_directory):
        print("La base de conocimiento no existe. Creando una nueva...")
        db_creada = crear_base_de_conocimiento_unificada()
        if db_creada is None:
            exit()
    else:
        print("La base de conocimiento ya existe. Omitiendo la creación y cargando la existente.")

    while True:
        pregunta = input("Ingresa tu pregunta (o 'salir' para terminar): ")
        if pregunta.lower() == "salir":
            break
        consultar_base_de_conocimiento(pregunta)